In [1]:
import sys
if ".." not in sys.path:
    sys.path.append("..")

import tensorflow as tf
# from tensorflow.python.keras import layers, losses, metrics, models

from src.constants import LEN_UCI_MOVES
from src.neural_network.actor import train_on_files as actor_train_files, evaluate_on_files as actor_eval_files
from src.neural_network.critic import train_on_files as critic_train_files, evaluate_on_files as critic_eval_files

activation = "relu"
actor_model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(70),
    tf.keras.layers.Dense(1000, activation=activation),
    tf.keras.layers.Dense(1000, activation=activation),
    tf.keras.layers.Dense(1000, activation=activation),
    tf.keras.layers.Dense(LEN_UCI_MOVES, activation="softmax")
])
actor_model.compile(
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = [tf.keras.metrics.CategoricalCrossentropy()],
)


critic_model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(70),
    tf.keras.layers.Dense(1000, activation=activation),
    tf.keras.layers.Dense(1000, activation=activation),
    tf.keras.layers.Dense(1000, activation=activation),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
critic_model.compile(
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = [tf.keras.metrics.BinaryCrossentropy()],
)

In [2]:
train_files = ["../data/split/processed/chess1.pkl"]
eval_files = ["../data/split/processed/chess1.pkl"]
actor_model = actor_train_files(actor_model, train_files, ckpt_file="../actor_model")

Training on ../data/split/processed/chess1.pkl
31250/31250 [==============================] - 238s 8ms/step - loss: 5.1414 - categorical_crossentropy: 5.1414
INFO:tensorflow:Assets written to: ../actor_model/assets


INFO:tensorflow:Assets written to: ../actor_model/assets


In [3]:
critic_model = critic_train_files(critic_model, train_files, ckpt_file="../critic_model")

Training on ../data/split/processed/chess1.pkl
31250/31250 [==============================] - 169s 5ms/step - loss: 0.6159 - binary_crossentropy: 0.6159
INFO:tensorflow:Assets written to: ../critic_model/assets


INFO:tensorflow:Assets written to: ../critic_model/assets


In [4]:
print(actor_eval_files(actor_model, eval_files))
print(critic_eval_files(critic_model, eval_files))

31250/31250 [==============================] - 52s 2ms/step
{'categorical_accuracy': 0.1380240023136139, 'categorical_crossentropy': 4.719549655914307}
31250/31250 [==============================] - 38s 1ms/step
{'binary_accuracy': 0.5867379903793335, 'binary_crossentropy': 0.5960864424705505}


In [2]:
critic_model = tf.keras.models.load_model("../ckpt/critic_model")

from src.neural_network.actor_model import ActorModel


def critic_loss_function(X):
    return critic_model.predict(X)

actor_model = ActorModel(critic_loss_function=critic_loss_function)
actor_model = actor_train_files(actor_model, ["../data/split/processed/chess1.pkl"])
print(actor_eval_files(actor_model, ["../data/split/processed/chess1.pkl"]))

Training on ../data/split/processed/chess1.pkl


RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.